In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_train=pd.read_csv('/content/drive/MyDrive/week6/Classification Datsets/Classification Datsets/Bank Client Deposit Data set Classification/Bank Client Deposit Data set Classification.csv')

In [ ]:
dataset_train.shape

In [ ]:
dataset_train.head()

In [ ]:
dataset_train.columns

In [ ]:
dataset_train.isnull().sum()

In [ ]:
dataset_train.describe(include='all')

In [ ]:
dataset_train.dtypes

In [ ]:
dataset_train.info()

In [ ]:
dataset_train.plot(kind='box',subplots=True,layout=(7,7),figsize=(30,30))
plt.show()
     

In [ ]:
dataset_train.plot(kind='box',subplots=True,layout=(7,7),figsize=(30,30))
plt.show()
     

In [ ]:
obj = (dataset_train.dtypes == 'object')
object_cols = list(obj[obj].index)
print("Categorical variables:",len(object_cols))
 
int_ = (dataset_train.dtypes == 'int')
num_cols = list(int_[int_].index)
print("Integer variables:",len(num_cols))

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(dataset_train.corr(), cmap="RdBu",annot=True)
plt.title("Correlations Between Variables", size=15)
plt.show() 

In [ ]:
num=dataset_train._get_numeric_data() 
num.head()

In [ ]:
dataset_train.nunique()

In [ ]:
df2_cat=dataset_train.drop(num,axis=1)
df2_cat.head()

In [ ]:
df=dataset_train.drop(['age','education','day','campaign','pdays','job'], axis=1)
df

In [ ]:
cat=dataset_train.drop(num,axis=1)
cat.head()

In [ ]:
dataset_train['marital'].unique() 

In [ ]:
dataset_train['default'].unique()

In [ ]:
df["y"] = df["y"].replace({"no": 0, "yes": 1})

In [ ]:

# Convert categorical columns into dummy variables
cat_cols = ['marital','default','housing','loan','contact','month','poutcome']
df_dummies = pd.get_dummies(dataset_train[cat_cols], drop_first=True)

# Concatenate the dummy variables with the original dataframe
df = pd.concat([df, df_dummies], axis=1)

# Drop the original categorical columns
df.drop(columns=cat_cols, inplace=True)

# Print the updated dataframe
df.head()


In [ ]:
df.shape

In [ ]:

class StandardScaler:
    def __init__(self):
        self.mean_ = None
        self.std_ = None
        
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        
    def transform(self, X):
        return (X - self.mean_) / self.std_
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)


In [ ]:
num = ['balance','duration']
X_train_num = df[num]

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training set
scaler.fit(X_train_num)

# Scale the numerical features in the training set
df[num] = scaler.transform(X_train_num)


In [ ]:
# Separate the target variable from the input features
y = df['y']
X = df.drop('y', axis=1) #axis=1 the operation is performed along the columns

In [ ]:
# Split the data into training and test sets
train_size = int(len(dataset_train) * 0.7) # 70% of the data is used for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Print the size of each set
print('Training data:', len(X_train))
print('test data:', len(X_test))

In [ ]:
X_train.shape

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

def binary_cross_entropy(y_true, y_pred):
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()


In [ ]:
class NeuralNetwork:
    def __init__(self, num_features):
        self.weights1 = np.random.rand(6, num_features)
        self.weights2 = np.random.rand(1, 6)
  

    def forward(self, X):
        self.hidden_layer = sigmoid(np.dot(X, self.weights1.T))
        self.output_layer = sigmoid(np.dot(self.hidden_layer, self.weights2.T))
        return self.output_layer
    
    def train(self, X_train, y_train, lr, epochs):
        y_train = y_train.values.reshape(-1, 1)  # convert to numpy array and reshape
        for i in range(epochs):
            # forward propagation
            output = self.forward(X_train)

            # calculate error
            error = y_train - output

            # backpropagation
            output_derivative = output * (1 - output)
            hidden_layer_derivative = self.hidden_layer * (1 - self.hidden_layer)
            delta2 = error * output_derivative
            delta1 = np.dot(delta2, self.weights2) * hidden_layer_derivative

            # update weights
            self.weights2 += lr * np.dot(delta2.T, self.hidden_layer)
            self.weights1 += lr * np.dot(delta1.T, X_train)



In [ ]:
nn = NeuralNetwork(num_features=24)
nn.train(X_train, y_train, lr=0.01, epochs=100)

In [ ]:
y_pred = nn.forward(X_train)

# convert predictions to binary (0 or 1) using a threshold value of 0.5
y_pred_binary = np.where(y_pred >= 0.5, 1, 0)

# compare predictions to actual labels and calculate accuracy
accuracy = np.mean(y_pred_binary == y_train.values.reshape(-1, 1))
print('Train accuracy:', accuracy)

In [ ]:
y_pred_t = nn.forward(X_test)

# convert predictions to binary (0 or 1) using a threshold value of 0.5
y_pred_binary_t = np.where(y_pred_t >= 0.5, 1, 0)

# compare predictions to actual labels and calculate accuracy
accuracy = np.mean(y_pred_binary_t == y_test.values.reshape(-1, 1))
print('Test accuracy:', accuracy)

In [ ]:
# assume X_test and y_test are the test dataset and labels
# predict labels using traine

# create confusion matrix
confusion_matrix = np.zeros((2, 2))
for i in range(len(y_test)):
    true_label = int(y_test.iloc[i])
    predicted_label = int(y_pred_binary_t[i][0])
    confusion_matrix[true_label][predicted_label] += 1

# print confusion matrix
print("Confusion matrix:")
print(confusion_matrix)


In [ ]:
import itertools

# define hyperparameters and their possible values to try out
learning_rates = [0.1, 0.01, 0.001]
epochs = [100,200,300]
hidden_layer_sizes = [40, 45, 30]

# create a list of all possible hyperparameter combinations
hyperparameter_combinations = list(itertools.product(learning_rates, epochs, hidden_layer_sizes))

# initialize variables to keep track of best hyperparameters and best accuracy
best_hyperparameters = None
best_accuracy = 0

# loop over all possible hyperparameter combinations
for lr, epoch, hidden_layer_size in hyperparameter_combinations:
    # train model with current hyperparameters
    nn = NeuralNetwork(num_features=24)
    nn.train(X_train, y_train, lr=lr, epochs=epoch)
    
    # make predictions on train data
    y_pred = nn.forward(X_train)
    y_pred_binary = np.where(y_pred >= 0.5, 1, 0)

    # calculate accuracy
    accuracy = np.mean(y_pred_binary == y_train.values.reshape(-1, 1))
    # make predictions on test data
    y_pred = nn.forward(X_test)
    y_pred_binary_t = np.where(y_pred >= 0.5, 1, 0)

    # calculate accuracy
    accuracy = np.mean(y_pred_binary_t == y_test.values.reshape(-1, 1))
    

    # if current accuracy is better than previous best, update best hyperparameters and best accuracy
    if accuracy > best_accuracy:
        best_hyperparameters = (lr, epoch, hidden_layer_size)
        best_accuracy = accuracy
    
print("Best hyperparameters:", best_hyperparameters)
print("Best test accuracy:", best_accuracy)
print("Best train accuracy:", best_accuracy)